In [ ]:
import json
from pathlib import Path
import pandas as pd

In [ ]:
results = Path("runs/openai_qa_250211/").glob("*.jsonl")

In [ ]:
def load_openai(file: Path) -> list[dict[str, str]]:
    with open(file, "r", encoding="utf-8") as f:
        content=f.read()
        data = content.split("\n")
        qas = []

        for i in range(0, len(data)-1, 8):
            case = "".join(data[i:i+8])
            qas.append(json.loads(case))
        return qas
    


def flatten(data_deepseek: list[dict[str, str]], data_openai: list[dict[str, str]]) -> dict[str, str]:

    assert len(data_deepseek) == len(data_openai)
    
    parsed = []
    for data_ds, data_oai in zip(data_deepseek, data_openai):
        qa = {}
        for k, v in data_ds.items():
            qa[f"ds_{k}"] = v
        for k, v in data_oai.items():
            qa[f"oai_{k}"] = v
        parsed.append(qa)
    return parsed




dfs = []
for i in range(1, 6):
    data_openai = load_openai(Path(f"runs/openai_qa_250211/{i}.jsonl"))
    data_deepseek = json.load(open(f"runs/qa_250211/{i}.json", "r"))
    df = pd.DataFrame(flatten(data_deepseek["qas"], data_openai))
    df["title"] = data_deepseek["screening"]['title']
    df["file_name"] = data_deepseek['file_name']
    dfs.append(df)

df = pd.concat(dfs)
df.to_csv("tmp/export_qa.csv", index=False)
